In [1]:
import pandas as pd
import numpy as np

#Load Dataset

In [4]:
random = pd.read_csv('/content/binary_random_reddit.csv')

#LSTM

In [5]:
random.head()

,Unnamed: 0,T1,T2,class
0,0,Child poverty causes a number of problems for ...,"Well, it's a change-my-view about changing vie...",1.0
1,1,I am German and my opinion is based on the fla...,Many (if not all) of the same states (with som...,1.0
2,2,"Today is Australia's National Holiday, celebra...",This week there will be a vote to decide wheth...,1.0
3,3,"Abortion, gun safety laws, school shootings, r...","So they don’t actually know what QAnon is, but...",1.0
4,4,"Now, he's a great guy, but politically... let'...",If intelligence can confirm Putin and his most...,1.0


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(random['T1']+ random['T2'], random['class'], test_size=0.2, random_state=42)


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, Embedding, LSTM, Dense, Dropout, Conv1D, MaxPooling1D, GlobalMaxPooling1D

In [10]:
# tokenization and padding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)
max_length = max([len(seq) for seq in X_train_seq])
max_length = max([len(seq) for seq in X_test_seq])
X_train_padded = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_padded = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

2804


In [11]:

# LSTM model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=max_length))
model.add(Conv1D(64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Conv1D(32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=3))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(Dropout(0.2))
model.add(MaxPooling1D(pool_size=5))
model.add(Dense(32, activation='tanh'))
model.add(GlobalMaxPooling1D())
model.add(Dense(2, activation='softmax'))  # 2 classes: generated by same method or not
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train the model
model.fit(X_train_padded, y_train, epochs=15, batch_size=32)


Epoch 1/15
107/107 [==============================] - 43s 249ms/step - loss: 0.5096 - accuracy: 0.7287
Epoch 2/15
107/107 [==============================] - 25s 240ms/step - loss: 0.3661 - accuracy: 0.8190
Epoch 3/15
107/107 [==============================] - 24s 223ms/step - loss: 0.2882 - accuracy: 0.8588
Epoch 4/15
107/107 [==============================] - 21s 198ms/step - loss: 0.2388 - accuracy: 0.8778
Epoch 5/15
107/107 [==============================] - 19s 173ms/step - loss: 0.2120 - accuracy: 0.9026
Epoch 6/15
107/107 [==============================] - 18s 168ms/step - loss: 0.2135 - accuracy: 0.8994
Epoch 7/15
107/107 [==============================] - 15s 138ms/step - loss: 0.1789 - accuracy: 0.9155
Epoch 8/15
107/107 [==============================] - 17s 159ms/step - loss: 0.1625 - accuracy: 0.9281
Epoch 9/15
107/107 [==============================] - 14s 130ms/step - loss: 0.1389 - accuracy: 0.9418
Epoch 10/15
107/107 [==============================] - 14s 131ms/step - l

In [12]:
# tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_test)
loss, acc = model.evaluate(X_test_padded, y_test)

27/27 [==============================] - 2s 25ms/step - loss: 0.4287 - accuracy: 0.8807


In [13]:
X_test_padded.shape

(855, 2804)

In [14]:
X_train_padded.shape

(3420, 2804)

In [15]:
preds = model.predict(X_test_padded)
# preds = np.argmax(preds, axis=-1)
preds[1]


27/27 [==============================] - 2s 23ms/step


array([9.9934024e-01, 6.5977778e-04], dtype=float32)

In [16]:
preds_new = []
for i in preds:
  if(i[0]>i[1]):
    preds_new.append(0.0)
  else:
    preds_new.append(1.0)

In [17]:
preds_new[1]

0.0

In [18]:
y_test_array = np.array(y_test)
y_test_array[1]

0.0

In [20]:
from sklearn.metrics import f1_score,precision_score, recall_score, accuracy_score
f1 = f1_score(y_test, preds_new)
print("Random Scores")
print("F1 score = ",f1)
precision = precision_score(y_test, preds_new)
recall = recall_score(y_test, preds_new)
print("Precision = ", precision)
print("Recall = ", recall)
print("Accuracy = ", accuracy_score(y_test, preds_new))

Random Scores
F1 score =  0.8338762214983714
Precision =  0.8677966101694915
Recall =  0.8025078369905956
Accuracy =  0.8807017543859649
